<a href="https://colab.research.google.com/github/senushidinara/LearnScape-AI/blob/main/Wolfram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from IPython.display import Image, display
import json

# ---------------------------------------------
# CONFIG — paste your Wolfram AppID
# ---------------------------------------------
API_KEY = "H34WX632E2"

# ---------------------------------------------
# MAIN FUNCTION — full processing pipeline
# ---------------------------------------------
def wolfram_all(query):
    print("🔍 Querying Wolfram Alpha...")

    # Build URL
    url = (
        f"https://api.wolframalpha.com/v2/query"
        f"?input={query}"
        f"&appid={API_KEY}"
        f"&output=json"
    )

    # Request JSON
    data = requests.get(url).json()

    # Prepare result container
    output = {
        "query": query,
        "pods": [],
        "images": [],
        "plots": [],
        "steps": [],
        "text_blocks": []
    }

    # Extract pods
    pods = data["queryresult"].get("pods", [])

    for pod in pods:
        title = pod["title"]
        subpod = pod["subpods"][0]

        # Save text content
        plaintext = subpod.get("plaintext", "")
        if plaintext:
            output["text_blocks"].append({"title": title, "text": plaintext})

        # Save images
        if "img" in subpod:
            img_url = subpod["img"]["src"]
            output["images"].append({"title": title, "url": img_url})

            # If it's a plot, store separately
            if "Plot" in title or "plot" in title.lower():
                output["plots"].append(img_url)

        # Step-by-step solutions
        if "Step-by-step solution" in title:
            output["steps"].append(plaintext)

        # Store raw pod
        output["pods"].append({
            "title": title,
            "plaintext": plaintext,
            "image": subpod.get("img", {}).get("src", None)
        })

    # Display images directly in Colab
    print("\n📊 Showing visual outputs:\n")
    for img in output["images"]:
        print("Image:", img["title"])
        display(Image(url=img["url"]))

    print("\n✅ Finished.")
    return output


# ---------------------------------------------
# RUN EVERYTHING WITH ONE COMMAND
# ---------------------------------------------
query = "definite integral of x^2 + 2x from 0 to 5"
result = wolfram_all(query)

# Show the cleaned LearnScape-ready dictionary
print("\n============= LEARNSCAPE OUTPUT =============\n")
print(json.dumps(result, indent=2))

🔍 Querying Wolfram Alpha...

📊 Showing visual outputs:

Image: Definite integral


Image: Visual representation of the integral


Image: Riemann sums


Image: Indefinite integral



✅ Finished.

============= LEARNSCAPE OUTPUT =============

{
  "query": "definite integral of x^2 + 2x from 0 to 5",
  "pods": [
    {
      "title": "Definite integral",
      "plaintext": "integral_0^5 x^2 2 x dx = 625/2 = 312.5",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq5ydkim5v.gif"
    },
    {
      "title": "Visual representation of the integral",
      "plaintext": "",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq661p7oc2.gif"
    },
    {
      "title": "Riemann sums",
      "plaintext": "left sum | (625 (n - 1)^2)/(2 n^2) = 625/2 - 625/n + O((1/n)^2)\n(assuming subintervals of equal length)",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq61xvg6cp.gif"
    },
    {
      "title": "Indefinite integral",
      "plaintext": "integral x^2 2 x dx = x^4/2 + constant",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq5zwgenc2.gif"
    }
  ],
  "images": [
    {
      "title": "Definite integral",
      "url": "https

In [6]:
from pyngrok import ngrok

# Make sure your local server is running on this port.
# For example, if you have a FastAPI app running on port 8000:
# import uvicorn
# from fastapi import FastAPI

# app = FastAPI()

# @app.get("/")
# async def root():
#     return {"message": "Hello World"}

# if __name__ == "__main__":
#     uvicorn.run(app, host="0.0.0.0", port=8000)

# Connect ngrok to the local server port
public_url = ngrok.connect(addr="8000")
print(f"ngrok tunnel is available at: {public_url}")

ngrok tunnel is available at: NgrokTunnel: "https://d9aebb496a11.ngrok-free.app" -> "http://localhost:8000"


In [5]:
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "35ZCvM6qV5iMFq413w8lEPFy98q_3xDPjMgPxxNSdizuxfLGf"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("ngrok authtoken has been set.")

ngrok authtoken has been set.


Now that your ngrok authtoken is set, you can proceed with using `ngrok.connect()` to expose your local server. For example, if you have a server running on port 8000, you can use:

```python
# This is an example, make sure your server is running on the specified port.
# For example, if you have a FastAPI app running on port 8000:
# import uvicorn
# from fastapi import FastAPI

# app = FastAPI()

# @app.get("/")
# async def root():
#     return {"message": "Hello World"}

# if __name__ == "__main__":
#     uvicorn.run(app, host="0.0.0.0", port=8000)

# Then, in another cell, after setting the authtoken:
# from pyngrok import ngrok
# public_url = ngrok.connect(addr="8000")
# print(f"ngrok tunnel is available at: {public_url}")
```

In [7]:
import requests
import json
from IPython.display import HTML, Image, display

# YOUR APP ID
APP_ID = "H34WX632E2"

def wolfram_query(query):
    """Send query to Wolfram Alpha Full Results API"""
    url = "https://api.wolframalpha.com/v2/query"

    params = {
        "input": query,
        "appid": APP_ID,
        "output": "JSON",         # important: return JSON not XML
        "format": "image,plaintext"
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data


def parse_wolfram_result(data):
    """Extract clean results from Wolfram JSON"""

    pods = data.get("queryresult", {}).get("pods", [])

    # Get the input interpretation directly from the queryresult if available
    input_interpretation = data.get("queryresult", {}).get("inputstring", "N/A")

    result = {
        "input_interpretation": input_interpretation,
        "plaintext_results": [],
        "images": [],
        "3d_data": None,
        "permalink": None # To store a link to the full WA page
    }

    # Try to construct a permalink to the Wolfram Alpha website
    if input_interpretation and input_interpretation != "N/A":
        result["permalink"] = f"https://www.wolframalpha.com/input/?i={requests.utils.quote(input_interpretation)}"

    for pod in pods:
        title = pod.get("title", "").lower()

        # Text results / formulas / steps
        if pod.get("subpods"):
            for sp in pod["subpods"]:
                if "plaintext" in sp and sp["plaintext"]:
                    result["plaintext_results"].append({
                        "title": pod.get("title"),
                        "value": sp["plaintext"]
                    })

                # Image results (plots)
                if "img" in sp and "src" in sp["img"]:
                    result["images"].append({"title": pod.get("title"), "url": sp["img"]["src"]})

        # Try extracting 3D data if available (though often it's just an image)
        if "3d" in title or "plot" in title:
            try:
                # Wolfram sometimes puts JSON 3D data inside "minput" or "moutput"
                # For now, we'll just store the pod, as direct interactive 3D data isn't easily consumable
                result["3d_data"] = pod
            except:
                pass

    return result

def display_wolfram_result(result):
    """Display clean results from Wolfram in a more colorful and functional way"""

    html_output = []
    html_output.append("<style>")
    html_output.append(".wolfram-container { font-family: sans-serif; border: 1px solid #ddd; border-radius: 8px; padding: 15px; margin-bottom: 20px; background-color: #f9f9f9; }")
    html_output.append(".wolfram-section-title { font-size: 1.2em; color: #333; margin-top: 15px; margin-bottom: 10px; border-bottom: 1px solid #eee; padding-bottom: 5px; }")
    html_output.append(".wolfram-item-title { font-weight: bold; color: #0056b3; margin-top: 10px; }")
    html_output.append(".wolfram-text-block { background-color: #fff; border: 1px solid #e0e0e0; padding: 10px; border-radius: 5px; margin-bottom: 8px; overflow-x: auto; }")
    html_output.append(".wolfram-image-container { text-align: center; margin: 10px 0; border: 1px solid #eee; background-color: #fff; padding: 5px; border-radius: 5px; }")
    html_output.append(".wolfram-image-container img { max-width: 100%; height: auto; display: block; margin: 0 auto; }")
    html_output.append(".wolfram-link a { color: #007bff; text-decoration: none; }")
    html_output.append(".wolfram-link a:hover { text-decoration: underline; }")
    html_output.append("</style>")

    html_output.append("<div class='wolfram-container'>")

    html_output.append(f"<div class='wolfram-section-title'>Input Interpretation:</div>")
    html_output.append(f"<div>{result['input_interpretation']}</div>")

    if result["plaintext_results"]:
        html_output.append("<div class='wolfram-section-title'>Text Results:</div>")
        for item in result["plaintext_results"]:
            html_output.append(f"<div class='wolfram-item-title'>{item['title']}:</div>")
            html_output.append(f"<div class='wolfram-text-block'><pre>{item['value']}</pre></div>")

    if result["images"]:
        html_output.append("<div class='wolfram-section-title'>Visualizations:</div>")
        for img_data in result["images"]:
            html_output.append(f"<div class='wolfram-image-container'>")
            html_output.append(f"<div class='wolfram-item-title'>{img_data['title']}</div>")
            html_output.append(f"<img src='{img_data['url']}' alt='{img_data['title']}' />")
            html_output.append(f"</div>")

    if result["permalink"]:
        html_output.append("<div class='wolfram-section-title'>More Functions & Interactive Results:</div>")
        html_output.append(f"<div class='wolfram-link'>For interactive plots, step-by-step solutions, and more details, visit the <a href='{result['permalink']}' target='_blank'>full Wolfram Alpha page</a>.</div>")

    html_output.append("</div>")

    display(HTML("".join(html_output)))


# -----------------------------
#        RUN EVERYTHING
# -----------------------------
query = "plot x^2 + 2x from -5 to 5 in 3D"
data = wolfram_query(query)
result = parse_wolfram_result(data)
display_wolfram_result(result)
